In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7240\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
I   load_graph('file_path',backend_class)：（不需要修改，直接使用）
    ppq有自己的图表示graph，需要先把onnx模型转化为自己的graph结构，具体细节在ppq.IR.base文件中。
    构建一个GraphBuilder，就是一个build函数，在字典图中，每个Onnx算子用Operation()来表示，同时算子的输入和输出
    也保存在列表中，再用变量字典来保存权重值，把onnx图中所有的参数全部放入到variables。
            src                         dest                 _input_vars         _output_vars
       [op1,op2,op3] --> Variabel --->[op3]               [var1,var2]-->>op --->[var3]
                        data,types
    graph.operations={'op_name':OP_CLASS}
    graph.variables = {'var_name':VAR_CLASS}
    operations和variables是相互连接的形式，实现图的连接 ：op-->var-->op-->var-->op
    预处理图节点，做一系列简单的优化工作，就是修改graph中的Operations，

    format_graph(graph)：对计算图进行预处理工作，GraphCommandProcessor 命令类，里面有对应的操作函数
    之后做图的简单优化Constant,Batchnorm 等算子去除，实现方法是责任链模式n中的参数确定做
    哪些修改调用每个命令里的函数责任链（设计模式）为
    GraphReplacer--》GraphFormatter--》GraphMerger，具体根据参数GraphCommandProcessor，ppq.core.common来执行
    替换一些算子      算子格式修改      融合conv+bn
        
        
    注：GraphCommandProcessor继承后，在_acceptable_command_types添加操作的类型，并实现每一个操作，在process中调用操作
        调用时修改format_graph函数把命令名称添加到调用中
       
        
II dispatch_graph：(不用修改)图切分与调度，他是中间流程，可以自动，也可以手动
    创建调度器，就是dispatch函数，不同平台实现不一样，函数通过图匹配方法（复杂），把每一个OP按照类型，设置他的量化平台，放入
    dipatch_table字典中{op.name: TargetPlatform.FP32}。
    量化时就是对于每一个算子，使用dispatch_table来确定算子的量化平台是什么。
    
III.创建量化器（不同平台不同）
    自定义量化器继承BaseQuantizer，要实现init_quantize_config函数，
    最后注册register_network_quantizer(MyQuantizer, platform=TargetPlatform.ACADEMIC_INT4)，
    注意，这个东西是绑定TargetPlatform.ACADEMIC_INT4参数的，不是直接调用的，里面也有一个责任链
    QuantableGraph--》GraphReplacer
    （量化算子          替换一些算子）
    对外接口函数是quantize(遍历调用quantize_operation('op_name'))
        （1）prequant pipeline： build_prequant_pipeline函数，（可以没有）
            创建一个QuantizationOptimizationPipeline（设计模式），是一个"量化管道"列表，将setting中的参数传递给
            QuantizationOptimizationPass实例中，实例要有一个optimize方法，再调用所有PASS的optimize
            #HorizontalLayerSplitPass，ChannelwiseSplitPass，LayerwiseEqualizationPass
        （2）为算子初始化量化信息
            根据OP的platform来决定是否量化，quantize_operation函数用QuantizeOperationCommand实现量化一个op，
            这里就是使用责任链的地方，执行QuantableGraph.quantize_operation，用一个QuantableOperation替换
            图中的Operation。
            
            自定义量化器继承BaseQuantizer，要实现init_quantize_config函数，创建参数字典，根据不同OP名称完成
            对应的量化参数的生成，给后续量化操作使用。
            
        （3）build_quant_pipeline量化，和（1）类似的"量化管道"，这个才是量化的执行步
            ParameterQuantizePass：给每一个OP添加一个observer((BaseTensorObserver)，根据setting参数选择量化方法
              'minmax': TorchMinMaxObserver,'kl': TorchHistObserver,'percentile': TorchPercentileObserver,
              'mse': TorchMSEObserver,'isotone': TorchIsotoneObserver,'constant': ConstantObserver,
              'floating': DirectMSEObserver
                
                render_quantization_config就是计算scale,offset参数的
            # RuntimeCalibrationPass：量化校准
            # QuantizeSimplifyPass - 用于移除网络中的冗余量化信息
            # QuantizeFusionPass - 用于调整量化信息状态，从而模拟推理图融合
optimize==  # ParameterQuantizePass - 用于为模型中的所有参数执行 Calibration, 生成它们的 scale，并将对应 TQC 的状态调整为 ACTIVED
            # RuntimeCalibrationPass - 用于为模型中的所有激活执行 Calibration, 生成它们的 scale，并将对应 TQC 的状态调整为 ACTIVED
            # QuantAlignmentPass - 用于执行 concat, add, sum, sub, pooling 算子的定点对齐
            # LearnedStepSizePass - 用于训练微调模型的权重，从而降低量化误差
            # ParameterBakingPass - 用于执行模型参数烘焙
            （自定义融合管线）
IV.BaseGraphExecutor(graph=ppq_quant_ir):PPQ 执行引擎 from ppq import TorchExecutor.__forward
      遍历graph，在图上执行计算，对于每一个算子他都有特定设备的计算方法函数，BaseGraphExecutor.tracing_operation_meta   
                                                     
V.可以直接用（2），（3）步进行手动量化，那就需要手动创建"量化管道"PFL.Pipeline，[pass.optime,pass.optim,pass.optim]
                                                     
    PASS的工作原理：遍历每一个可量化OP，创建{"op_name":OP_OBSERVER}，最后再渲染保存.
    所以，对于每一个算子的量化，依赖OP算子自己，还有OP的观察者OBserver。
                                                                                         
                                                     
VI 最后需要将graph再次导出成Onnx格式的文件，不同框架有不同的导出函数GraphExporter.export。就是遍历graph中所有的OP，每个OP
    都要创建一个Node,最终创建onnx格式的文件，流程大致一样，只是有些框架内细节要求不一样，需要修改而已。
    exporter = PFL.Exporter(platform=TargetPlatform.TRT_FP8)
    exporter.export(file_path='Quantized.onnx', graph=graph)#原地修改图，如果有需要就要复制graph                                         

QuantableOperation
相比于普通算子，一个量化算子具有以下额外的功能

1. 每一个量化算子都将具有一个 config(OperationQuantizationConfig) 属性PPQ 使用这个东西描述量化细节，在整个网络中，
    有且只有这一个量化表示executor, dispatcher, optimization pass, exporter都是围绕这一属性工作的

2. 每一个量化算子都将有一个 dequantize 方法和 restore_quantize_state 方法，一旦一个量化算子被 dequantize() 方法解除量化，
   该算子的 OperationQuantizationConfig 将被修改状态从而使得该算子的输入输出量化被暂时停用。被解除量化的算子可以随时通过 
   restore_quantize_state 方法恢复量化状态对一个算子多次重复执行 dequantize 是可以的

3. 每一个量化算子都将有一个 baking parameter 方法当算子具有有效的量化参数时，baking_parameters() 
   方法将对该算子的参数执行静态量化一旦静态量化完成，算子参数将被量化后的值替换；同时 config 的状态将被设置为: baked

4. 每一个量化算子都将有一个 store_parameter_value 方法该方法将算子目前的参数保存入缓存；PPQ 将在创建 QuantableOperation 
    时执行此函数从而保存算子的原始参数，以备后续取用。
   一个显而易见的例子是，一旦算子执行了 baking_parameters 方法，它的参数值将被修改，此时若要完全还原算子状态，
   需要从缓存中取出算子的原始参数，并替换当前的值当你调用 restore_quantize_state 时，
   该方法会从缓存中取回保存的参数值并执行替换。你不应当手动调用该方法，该方法将影响到 PPQ 的核心逻辑正确性

5. 一个量化算子是可拷贝的，该拷贝只会拷贝算子的基本信息以及绑定的 OperationQuantizationConfig

OperationObserver：（设计模式）这是一个算子的，每个算子的相连的变量会创建一个BaseTensorObserver，再遍历每一个变量
    （根据给定算法选择类型），每个可量化算子都有一个，但不是同一个类
    BaseTensorObserver.observe(tensor)：就是观察张量里的统计量是什么
    BaseTensorObserver.render_quantization_config():有了统计量后要开始更新OP里面的config

# API.interface
    所有的需要函数都在这
    load_graph：先将该格式文件转化为自带的graph形式。
    quantize_torch_model（）
    quantize(working_directory: str, 
             setting: QuantizationSetting,
             model_type: NetworkFramework,#模型类型
             executing_device: str,
             input_shape: List[int], 
             target_platform: TargetPlatform,
             dataloader: DataLoader, 
             calib_steps: int = 32) 直接量化给定的模型的完整流程，返回图，保存Onnx格式文件

    export(working_directory: str,
        quantized: BaseGraph,
        platform: TargetPlatform, 
        **kwargs)：导出生成的图
        
最终生成的仍然是ONNX格式的文件，不过这个文件是已经适应了后端的ONNX格式文件，可以直接用于指定的后端。


最好的使用方法放在了samples中的best_samples中

In [ ]:

                        1.将图载入，转化中graph


可以直接调用这个函数
quantize_torch_model--》2.进行图切割，可以自定义划分节点是否进行量化，让每个算子都有platform属性


                                                                  3.1 预量化管道构建 ，optimize 

                        3.创建量化器，进行量化 quantize函数----》 3.2 遍历算子，调用quantize_operation创建量化参数
          
                                                                3.3 .构建量化管道，optimize

# Core
## quant 
    量化控制结构体 TensorQuantizationConfig 
    可序列化的参数类，里面包含了量化时的属性，比如量化策略，线性量化与相关属性等
    - Quantizer 负责初始化 TQC
    - Optim Passes 负责调整 TQC 的参数和状态
    - Exporter 负责导出 TQC
    
    多次量化只为了调参，找到最优的TQC

In [ ]:
from ppq import TensorQuantizationConfig as TQC
MyTQC = TQC(
    policy = QuantizationPolicy(
        QuantizationProperty.SYMMETRICAL + 
        QuantizationProperty.FLOATING +
        QuantizationProperty.PER_TENSOR + 
        QuantizationProperty.POWER_OF_2),
    rounding=RoundingPolicy.ROUND_HALF_EVEN,
    num_of_bits=8, quant_min=-448.0, quant_max=448.0, 
    exponent_bits=3, channel_axis=None,
    observer_algorithm='minmax'
)

In [ ]:
# 分析函数
from ppq import TargetPlatform, graphwise_error_analyse

from ppq.quantization.analyse.graphwise import statistical_analyse
from ppq.quantization.analyse.layerwise import layerwise_error_analyse
CALIBRATION=[torch.rand(size=(32,3,224,224)) for _ in range(32)]
graphwise_error_analyse(graph=graph, 
                        running_device='cuda', 
                        dataloader=CALIBRATION, 
                        collate_fn=collate_fn)
layerwise_error_analyse(graph=quantized, 
                         running_device=DEVICE, 
                         collate_fn=collate_fn,
                        dataloader=CALIBRATION)
statistical_analyse(graph=quantized, 
                    running_device=DEVICE, 
                    collate_fn=collate_fn, 
                    dataloader=CALIBRATION)#更加细致

# QAT 
    是一个单独的训练过程，掉包实现在samples.QAT文件中